# Pre-processing Raw Influent Data - Calumet Water Reclamation Plant 

Sources: https://mwrd.org/raw-influents (Calumet Plant)

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import Influent Data and Refine Range of Dates

In [16]:
influent_1 = pd.read_excel(r"C:\Users\18455\EAEE4000_ML_Jupyter\EAEE4000_WW_Prediction\Appendix_A_Preprocessing_and_DataReview\Calumet Raw 2001- 2010.xls", sheet_name = None)
influent_2 = pd.read_excel(r"C:\Users\18455\EAEE4000_ML_Jupyter\EAEE4000_WW_Prediction\Appendix_A_Preprocessing_and_DataReview\Calumet_Raw_2011-2020.xlsx", sheet_name = None)

In [17]:
all_sheets = []
for name, sheet in influent_1.items():
    sheet['sheet'] = name
    sheet = sheet.rename(columns=lambda x: x.split('\n')[-1])
    all_sheets.append(sheet)

influent_set1 = pd.concat(all_sheets)
influent_set1.reset_index(inplace=True, drop=True)
influent_set1.rename(columns = {'Unnamed: 1':'pH','NO2-N + NO3-N':'Nitrates' }, inplace = True)
influent_set1.columns.values[0]='Date'

In [18]:
all_sheets = []
for name, sheet in influent_2.items():
    sheet['sheet'] = name
    sheet = sheet.rename(columns=lambda x: x.split('\n')[-1])
    all_sheets.append(sheet)

influent_set2 = pd.concat(all_sheets)
influent_set2.reset_index(inplace=True, drop=True)
influent_set2.rename(columns = {'Unnamed: 1':'pH', 'NO2-N + NO3-N':'Nitrates'}, inplace = True)
influent_set2.columns.values[0]='Date'

In [19]:
influent_data = pd.concat([influent_set1, influent_set2], ignore_index=True)

# Refine Raw Influent Dataset Characteristics and Convert Dates

In [20]:
influent_data = influent_data[['Date','BOD5','CBOD5','SS','VSS', 'TS','VTS','TKN','Nitrates','P-TOT','P-SOL']]

In [21]:
dates = pd.to_datetime(influent_data['Date'], errors='coerce')
nums = pd.to_numeric(influent_data['Date'], errors='coerce')
influent_data = influent_data[(dates.notnull()) | (nums.notnull())]

In [22]:
influent_data["Date"] = influent_data["Date"].fillna(0)
influent_data = influent_data.loc[influent_data["Date"] != 0]

In [23]:
influent_data = influent_data.loc[(influent_data['Date'] > '2007-12-31') & (influent_data['Date'] < '2020-01-01')]

In [24]:
influent_data["Date"] = influent_data["Date"].astype('datetime64[ns]')
influent_data["Date"] = pd.to_datetime(influent_data["Date"]).dt.date

# Clean Input Data for Model Use

In [25]:
influent_data = influent_data[['Date','BOD5','CBOD5','SS','VSS', 'TS','VTS','TKN','Nitrates','P-TOT','P-SOL']]  

In [26]:
# Data cleaning assumption - Replace any < or > with zero
influent_data['Nitrates'].replace({'     <0.02':0.02}, inplace=True)
influent_data['Nitrates'].replace({'<0.1':0.01}, inplace=True)
influent_data['Nitrates'].replace({'<0.15':0.15}, inplace=True)
influent_data['Nitrates'].replace({'     <0.15':0.15}, inplace=True)
influent_data['Nitrates'].replace({'     <0.25':0.15}, inplace=True)

influent_data['Nitrates'].replace({'       S/X':None}, inplace=True)
influent_data['Nitrates'] = influent_data['Nitrates'].fillna(0)

influent_data['Nitrates'] = influent_data['Nitrates'].astype(float)

influent_data['VSS'].replace({'<8':8}, inplace=True)
influent_data['P-SOL'].replace({'     <0.15':0.15}, inplace=True)

In [27]:
# Data cleaning assumption - Replace any T/X, N/S, S/X, N/A with zero
# For loop was tried, but columns each had different spacing and text values requiring manual review of what to be replaced
influent_data['CBOD5'].replace({'      T/X':None}, inplace=True)
influent_data['CBOD5'].replace({'   T/X':None}, inplace=True)
influent_data['CBOD5'].replace({'T/X':None}, inplace=True)
influent_data['CBOD5'].replace({'      S/X':None}, inplace=True)
influent_data['CBOD5'].replace({'      N/A':None}, inplace=True)
influent_data['CBOD5'].replace({'      N/S':None}, inplace=True)
influent_data['CBOD5'] = influent_data['CBOD5'].fillna(0)

influent_data['BOD5'].replace({'      T/X':None}, inplace=True)
influent_data['BOD5'].replace({'   T/X':None}, inplace=True)
influent_data['BOD5'].replace({'T/X':None}, inplace=True)
influent_data['BOD5'].replace({'      S/X':None}, inplace=True)
influent_data['BOD5'].replace({'      N/A':None}, inplace=True)
influent_data['BOD5'].replace({'      N/S':None}, inplace=True)
influent_data['BOD5'] = influent_data['BOD5'].fillna(0)

influent_data['SS'].replace({'      T/X':None}, inplace=True)
influent_data['SS'].replace({'   T/X':None}, inplace=True)
influent_data['SS'].replace({'T/X':None}, inplace=True)
influent_data['SS'].replace({'      S/X':None}, inplace=True)
influent_data['SS'].replace({'      N/A':None}, inplace=True)
influent_data['SS'].replace({'      N/S':None}, inplace=True)
influent_data['SS'] = influent_data['SS'].fillna(0)

influent_data['VSS'].replace({'      T/X':None}, inplace=True)
influent_data['VSS'].replace({'   T/X':None}, inplace=True)
influent_data['VSS'].replace({'T/X':None}, inplace=True)
influent_data['VSS'].replace({'      S/X':None}, inplace=True)
influent_data['VSS'].replace({'      N/A':None}, inplace=True)
influent_data['VSS'].replace({'      N/S':None}, inplace=True)
influent_data['VSS'] = influent_data['VSS'].fillna(0)

influent_data['VTS'].replace({'   T/X':None}, inplace=True)
influent_data['VTS'].replace({'T/X':None}, inplace=True)
influent_data['VTS'].replace({'      S/X':None}, inplace=True)
influent_data['VTS'].replace({'      N/A':None}, inplace=True)
influent_data['VTS'].replace({'      N/S':None}, inplace=True)
influent_data['VTS'] = influent_data['VTS'].fillna(0)

influent_data['TS'].replace({'      T/X':None}, inplace=True)
influent_data['TS'].replace({'   T/X':None}, inplace=True)
influent_data['TS'].replace({'T/X':None}, inplace=True)
influent_data['TS'].replace({'      S/X':None}, inplace=True)
influent_data['TS'].replace({'      N/A':None}, inplace=True)
influent_data['TS'].replace({'      N/S':None}, inplace=True)
influent_data['TS'] = influent_data['TS'].fillna(0)

influent_data['P-TOT'].replace({'      T/X':None}, inplace=True)
influent_data['P-TOT'].replace({'   T/X':None}, inplace=True)
influent_data['P-TOT'].replace({'T/X':None}, inplace=True)
influent_data['P-TOT'].replace({'       T/X':None}, inplace=True)
influent_data['P-TOT'].replace({'      S/X':None}, inplace=True)
influent_data['P-TOT'].replace({'       S/X':None}, inplace=True)
influent_data['P-TOT'].replace({'      N/A':None}, inplace=True)
influent_data['P-TOT'].replace({'      N/S':None}, inplace=True)
influent_data['P-TOT'] = influent_data['P-TOT'].fillna(0)

influent_data['P-SOL'].replace({'      T/X':None}, inplace=True)
influent_data['P-SOL'].replace({'   T/X':None}, inplace=True)
influent_data['P-SOL'].replace({'       T/X':None}, inplace=True)
influent_data['P-SOL'].replace({'T/X':None}, inplace=True)
influent_data['P-SOL'].replace({'      S/X':None}, inplace=True)
influent_data['P-SOL'].replace({'       S/X':None}, inplace=True)
influent_data['P-SOL'].replace({'      N/A':None}, inplace=True)
influent_data['P-SOL'].replace({'      N/S':None}, inplace=True)
influent_data['P-SOL'] = influent_data['P-SOL'].fillna(0)

influent_data['TKN'].replace({'      T/X':None}, inplace=True)
influent_data['TKN'].replace({'   T/X':None}, inplace=True)
influent_data['TKN'].replace({'T/X':None}, inplace=True)
influent_data['TKN'].replace({'      S/X':None}, inplace=True)
influent_data['TKN'].replace({'      N/A':None}, inplace=True)
influent_data['TKN'].replace({'      N/S':None}, inplace=True)
influent_data['TKN'] = influent_data['TKN'].fillna(0)

influent_data['Nitrates'].replace({'      T/X':None}, inplace=True)
influent_data['Nitrates'].replace({'   T/X':None}, inplace=True)
influent_data['Nitrates'].replace({'T/X':None}, inplace=True)
influent_data['Nitrates'].replace({'      S/X':None}, inplace=True)
influent_data['Nitrates'].replace({'      N/A':None}, inplace=True)
influent_data['Nitrates'].replace({'      N/S':None}, inplace=True)
influent_data['Nitrates'] = influent_data['Nitrates'].fillna(0)

# Export Pre-processed Data

In [28]:
influent_data.to_excel('raw_influent.xlsx')